# Testing

In [1]:
!python --version

Python 3.9.22


In [2]:
import os
import geo_mapper_api.converters as gma
import osmnx as ox
import geopandas as gpd
import geo_mapper_api.filters as filters

In [3]:
import pandas as pd
print(f"pandas version: {pd.__version__}")
print(f"osmnx version: {ox.__version__}")
print(f"geopandas version: {gpd.__version__}")
import fiona as fiona
print(f"fiona version: {fiona.__version__}")

pandas version: 1.4.4
osmnx version: 1.9.4
geopandas version: 0.12.2
fiona version: 1.9.6


In [4]:
DATA_DIR = "./data"
geojson_path = os.path.join(DATA_DIR, 'maprelease-geojson/USA_Tennessee.geojson')
csv_path = os.path.join(DATA_DIR, 'maprelease-osmconflation/USA_Tennessee.csv')
county_name = 'HAYWOOD'

In [21]:
df = gpd.read_file(geojson_path)
df = df[df['County'] == 'CROCKETT']

In [ ]:
osmnx_inrix = pd.read_csv(csv_path)
osmnx_inrix['XDSegID'] = osmnx_inrix['XDSegID'].astype(str)
tdf = osmnx_inrix[osmnx_inrix.XDSegID.isin(df['XDSegID'])].reset_index(drop=True)


,XDSegID,OSMWayIDs,OSMWayDirections,WayStartOffset_m,WayEndOffset_m,WayStartOffset_percent,WayEndOffset_percent
0,156331755,19440068,N,41.55,278.76,14.905,100.000
1,156481984,19440068,N,0.01,41.55,0.004,14.905
2,156496498,19439192;19553227,N;N,0.01,1213.82,0.001,100.000
3,156926265,19439237,N,1587.48,2751.71,57.665,99.955
4,159602557,19439237,P,2311.44,2751.74,83.962,99.956
...,...,...,...,...,...,...,...
389,1524644417,255386268,P,595.74,1753.94,14.451,42.544
390,1524644418,147599179;255386268,P;P,10711.15,595.74,97.113,14.451
391,1524644419,255386268,P,2582.00,3893.52,62.630,94.443
392,1524644420,255386268,P,1753.94,2582.00,42.544,62.630


In [23]:
tdf['XDSegID'] = tdf['XDSegID'].astype(int)
tdf.to_csv("./tests/data/test.csv", index=False)

In [ ]:
haywood_inrix_df = gma.load_and_prepare_inrix_data(geojson_path, county_name)
haywood_inrix_df.head(1)

In [5]:
print(haywood_inrix_df.iloc[0].tolist())

['7931440', 156418860, nan, 395960459.0, '4', nan, 'DR HESS RD', nan, 'UNITED STATES', 'TENNESSEE', 'HAYWOOD', nan, '38006', '0.5902665205613952', '1.0', '0', nan, 'DR HESS RD', '35.67248', '-89.14147', '35.666218484838986', '-89.13571015096953', 'S', '2013963', '4326', <LINESTRING (-89.141 35.672, -89.141 35.672, -89.141 35.672, -89.141 35.672,...>]


In [6]:
# Get bounding box
bbox = haywood_inrix_df.exterior.bounds

In [7]:
# Get OSM graph
polygon= gma.get_exterior_polygon(haywood_inrix_df)
G = gma.get_osm_graph(polygon, filters.base_filter)
# Get nodes and edges
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
# Prepare OSMNX INRIX data
haywood_osm = gma.prepare_osmnx_inrix_data(csv_path, haywood_inrix_df)
# Merge and filter INRIX and OSM data
merged_osm_inrix = gma.merge_and_filter_inrix_osm(edges, haywood_osm, haywood_inrix_df)

# Apply bearing classification
merged_osm_inrix  = gma.apply_bearing_classification(merged_osm_inrix)

# Filter groups by match
filtered_osm_inrix, unfiltered__osm_inrix = gma.filter_groups_by_match(merged_osm_inrix)
# Plot the results
gma.plot_matched_geometry(filtered_osm_inrix ,'geometry_osm', 'Matched Geometry ')
gma.plot_matched_geometry(merged_osm_inrix, 'geometry_inrix', 'Inrix Geometry ')
gma.plot_matched_geometry(edges, 'geometry', 'OSM Geometry ')


In [8]:
merged_osm_inrix.columns

Index(['u', 'v', 'key', 'osmid', 'lanes', 'name', 'highway', 'oneway',
       'reversed', 'length', 'geometry_osm', 'XDSegID', 'OSMWayDirections',
       'OID', 'PreviousXD', 'NextXDSegI', 'FRC', 'RoadNumber', 'RoadName',
       'LinearID', 'Country', 'State', 'County', 'District', 'PostalCode',
       'Miles', 'Lanes', 'SlipRoad', 'SpecialRoa', 'RoadList', 'StartLat',
       'StartLong', 'EndLat', 'EndLong', 'Bearing', 'XDGroup', 'ShapeSRID',
       'geometry_inrix', 'osm_line_bearing', 'compass_direction', 'match',
       'geometry'],
      dtype='object')

In [9]:

# filtered_osm_inrix.to_csv('../data/processed_osm_mapping/filtered_osm_inrix.csv', index=False)


# Group by 'u', 'v', and 'geometry'
grouped = filtered_osm_inrix.groupby(['u', 'v','key'])

# Process the top groups and find overlapping geometries
all_overlapping_geometries = gma.process_top_groups(grouped, nodes)

# Update the original DataFrame with the results
# filtered_osm_inrix = pd.merge(filtered_osm_inrix, all_overlapping_geometries, on=['u', 'v', 'osmid', 'XDSegID', 'geometry'], how='left', suffixes=('', '_updated'))


# Reproject both geometry columns to a metric CRS (e.g., EPSG 3857 for Web Mercator)
merged_osm_inrix = gpd.GeoDataFrame(all_overlapping_geometries.copy(),geometry='geometry')
merged_osm_inrix = gma.reproject_geometry(merged_osm_inrix, 'geometry')
merged_osm_inrix = gma.reproject_geometry(merged_osm_inrix, 'geometry_inrix')

# Define the acceptable threshold distance in meters
threshold_distance = 25.0  # 5 meters

# Calculate and check distances
proximity_results = gma.check_osm_inrix_proximity(merged_osm_inrix, threshold_distance)

# # Identify and print outliers
# outliers = proximity_results[~proximity_results['within_threshold']]
# print(f"Found {len(outliers)} outliers:")
# print(outliers)

# # Save results to CSV
# proximity_results.to_csv('osm_inrix_proximity_results.csv', index=False)

# Identify and print outliers
outliers = proximity_results[~proximity_results['within_threshold']]

prox = proximity_results[proximity_results['within_threshold']]
# prox.to_csv('../data/processed_osm_mapping/processed_osm_inrix_mapping.csv', index=False)

In [10]:
prox.columns

Index(['u', 'v', 'key', 'osmid', 'XDSegID', 'osm_geom', 'inrix_geom',
       'distance', 'within_threshold'],
      dtype='object')

In [11]:
prox

,u,v,key,osmid,XDSegID,osm_geom,inrix_geom,distance,within_threshold
0,202619796,202705554,0,19495638,1524404117,LINESTRING (-9927715.889573382 4219197.2535342...,LINESTRING (-9927119.795074416 4219027.3303308...,0.000000,True
1,202619796,202705554,0,19506479,1524404117,LINESTRING (-9927715.889573382 4219197.2535342...,LINESTRING (-9927119.795074416 4219027.3303308...,0.000000,True
2,202660241,202695030,0,19505111,1524613134,LINESTRING (-9951811.460897446 4226780.4570780...,LINESTRING (-9952156.840749582 4226892.3504523...,0.000000,True
3,202660241,202695034,0,19505111,1524613067,LINESTRING (-9951811.460897446 4226780.4570780...,LINESTRING (-9951080.154774534 4226265.8523310...,0.000000,True
4,202660282,202701592,0,132956177,1524311891,LINESTRING (-9948555.53277098 4244355.24504949...,LINESTRING (-9949140.033328388 4244245.1180524...,9.872074,True
...,...,...,...,...,...,...,...,...,...
1304,11080826555,457621929,0,107475220,441440153,LINESTRING (-9950863.24147487 4219174.00812974...,LINESTRING (-9950883.737007204 4219158.5047289...,0.000000,True
1305,11118621239,202660422,0,19500798,1524348043,LINESTRING (-9926553.157492047 4252924.9734564...,LINESTRING (-9926199.1101155 4253186.601635585...,3.899724,True
1306,11280287285,202698218,0,264911872,449573054,LINESTRING (-9936595.288756508 4244871.3884543...,LINESTRING (-9936613.74552808 4244872.55207690...,0.000000,True
1307,12343138953,202698986,0,19506151,449573052,LINESTRING (-9936630.365528056 4244433.1085382...,LINESTRING (-9936622.400771953 4244729.4777377...,0.357817,True
